In [1]:
# The C2QA pacakge is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Bose Hubbard model

In [4]:
import c2qa
import qiskit
import numpy as np
import scipy
import itertools
import c2qa.util as util
import evo
import c2qa.stateReadout as stateReadout
import matplotlib.pyplot as plt
import matplotlib
# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.providers.aer import AerSimulator
from qiskit import IBMQ
from collections import Counter

# Create a circuit with modes

In [54]:
numberofmodes=2
numberofqubitspermode=2
cutoff=2**numberofqubitspermode
qmr = c2qa.QumodeRegister(num_qumodes=numberofmodes, num_qubits_per_qumode=numberofqubitspermode)
circuit = c2qa.CVCircuit(qmr)

diffstallmodes = [1, 0]
for i in range(qmr.num_qumodes):
    circuit.cv_initialize(diffstallmodes[i], qmr[i])

In [56]:
dt=1
J=1
U=1
N=5

stateop, _ = c2qa.util.simulate(circuit)
print("Initial state ")
util.stateread(stateop, 0, numberofmodes, cutoff)
print("At each time step")
for i in range(N):
    print("dt+1")
    evo.eiht(circuit, qmr[0], qmr[1], J, U, dt)
    stateop, _ = c2qa.util.simulate(circuit)
    util.stateread(stateop, 0, numberofmodes, cutoff)

Initial state 
qumodes:  01  qubits:  0     with amplitude:  0.7071067811865476
qumodes:  01  qubits:  1     with amplitude:  0.7071067811865475
occupation modes  [0.0, 1.0]
occupation qubits  [0.4999999999999999]
At each time step
dt+1
qumodes:  10  qubits:  0     with amplitude:  -0.6205445805637456
qumodes:  10  qubits:  1     with amplitude:  0.6205445805637455
occupation modes  [0.7701511529340699, 0.0]
occupation qubits  [0.38507557646703483]
dt+1
qumodes:  01  qubits:  0     with amplitude:  -0.7071067811865476
qumodes:  01  qubits:  1     with amplitude:  -0.7071067811865475
occupation modes  [0.0, 1.0]
occupation qubits  [0.4999999999999999]
dt+1
qumodes:  10  qubits:  0     with amplitude:  0.6205445805637456
qumodes:  10  qubits:  1     with amplitude:  -0.6205445805637455
occupation modes  [0.7701511529340699, 0.0]
occupation qubits  [0.38507557646703483]
dt+1
qumodes:  01  qubits:  0     with amplitude:  0.7071067811865476
qumodes:  01  qubits:  1     with amplitude:  0.70